#                        Random ANN generation and training                                    <img src="logo-orange.png" />

### 1.    Pre-processing phase: with the use of unsupervised learning to label the data

In [1]:
#import keras
import pandas as pd
import numpy as np
import ast
import matplotlib
import matplotlib.pyplot as plt
import os
#matplotlib.style.use('ggplot')
%matplotlib inline
from Util import preprocessing, set_dummies, activ, init, layers, optim, structure, scores
import seaborn as sns
matplotlib.rcParams.update({'font.size': 22})
sns.set(font_scale=1.5)

Using TensorFlow backend.


p = [0.12857142857142856, 0.12857142857142856, 0.11428571428571428, 0.11428571428571428, 0.1, 0.1, 0.08571428571428572, 0.08571428571428572, 0.07142857142857142, 0.07142857142857142] 
 Using Keras version 2.1.6, backended with tensorflow 

 The number of models = 10 , the max number of layer is 20.


### Loading the dataset

In [2]:
dframe = pd.read_csv("/home/jaafar/Desktop/TNSM/data/Tmp/X_126.csv",delimiter=';',header='infer',index_col=[0],nrows=1000)

FileNotFoundError: File b'/home/jaafar/Desktop/TNSM/data/Tmp/X_126.csv' does not exist

In [27]:
dframe.head()

,ellis-cpu.idle_perc.csv,ralf-load.avg_15_min.csv,homer-net.out_bytes_sec.csv,ellis-io.write_req_sec.csv,homestead-load.avg_1_min.csv,homer-load.avg_1_min.csv,ralf-cpu.idle_perc.csv,sprout-io.read_req_sec.csv,homestead-net.in_bytes_sec.csv,homer-disk.space_used_perc.csv,...,sprout-mem.free_mb.csv,homer-io.write_req_sec.csv,ellis-disk.space_used_perc.csv,sprout-net.out_packets_sec.csv,ralf-cpu.system_perc.csv,ralf-mem.free_mb.csv,bono-io.read_req_sec.csv,bono-net.in_packets_sec.csv,homestead-cpu.wait_perc.csv,ellis-load.avg_1_min.csv
0,7.155661,6.024735,2.956697,1.487579,1.493882,5.052356,8.517342,1.0,1.794528,1.0,...,8.707317,5.274451,1.642857,1.441577,1.274112,7.513460,1.0,1.172696,4.207447,7.034884
1,7.728775,6.342756,2.946119,1.500860,1.367074,3.450262,8.582371,1.0,1.791270,1.0,...,8.711382,5.274451,1.642857,1.428249,1.274112,7.508549,1.0,1.173083,3.058511,7.244186
2,7.410378,6.501766,2.962111,1.490639,1.387097,2.790576,8.517342,1.0,1.798901,1.0,...,8.709349,5.104075,1.642857,1.440900,1.319797,7.511823,1.0,1.173375,2.723404,6.302326
3,7.176886,6.448763,2.950903,1.483509,1.273637,2.649215,8.634393,1.0,1.799224,1.0,...,8.709349,5.206250,1.642857,1.426670,1.228426,7.500364,1.0,1.170270,2.723404,5.988372
4,7.346699,6.501766,2.954983,1.483509,1.233593,2.696335,8.322255,1.0,1.786141,1.0,...,8.707317,5.316853,1.642857,1.441347,1.365482,7.502001,1.0,1.182598,2.675532,6.023256


### The data set normalized distribution


<img src="Figures/metrics_boxplot.png" />

### use K-means clustering to label the data, k=3 means 3 classes

In [28]:
from sklearn.cluster import KMeans
import numpy as np
X = dframe.iloc[:,0:126].values
kmeans = KMeans(n_clusters=3).fit(X)

print set(kmeans.labels_)
# 3 SLO classes

set([0, 1, 2])


### Labeling of data 

In [29]:
dframe['SLA'] = kmeans.predict(dframe.iloc[:,0:126])

In [30]:
dframe.head()

,ellis-cpu.idle_perc.csv,ralf-load.avg_15_min.csv,homer-net.out_bytes_sec.csv,ellis-io.write_req_sec.csv,homestead-load.avg_1_min.csv,homer-load.avg_1_min.csv,ralf-cpu.idle_perc.csv,sprout-io.read_req_sec.csv,homestead-net.in_bytes_sec.csv,homer-disk.space_used_perc.csv,...,homer-io.write_req_sec.csv,ellis-disk.space_used_perc.csv,sprout-net.out_packets_sec.csv,ralf-cpu.system_perc.csv,ralf-mem.free_mb.csv,bono-io.read_req_sec.csv,bono-net.in_packets_sec.csv,homestead-cpu.wait_perc.csv,ellis-load.avg_1_min.csv,SLA
0,7.155661,6.024735,2.956697,1.487579,1.493882,5.052356,8.517342,1.0,1.794528,1.0,...,5.274451,1.642857,1.441577,1.274112,7.513460,1.0,1.172696,4.207447,7.034884,1
1,7.728775,6.342756,2.946119,1.500860,1.367074,3.450262,8.582371,1.0,1.791270,1.0,...,5.274451,1.642857,1.428249,1.274112,7.508549,1.0,1.173083,3.058511,7.244186,1
2,7.410378,6.501766,2.962111,1.490639,1.387097,2.790576,8.517342,1.0,1.798901,1.0,...,5.104075,1.642857,1.440900,1.319797,7.511823,1.0,1.173375,2.723404,6.302326,1
3,7.176886,6.448763,2.950903,1.483509,1.273637,2.649215,8.634393,1.0,1.799224,1.0,...,5.206250,1.642857,1.426670,1.228426,7.500364,1.0,1.170270,2.723404,5.988372,1
4,7.346699,6.501766,2.954983,1.483509,1.233593,2.696335,8.322255,1.0,1.786141,1.0,...,5.316853,1.642857,1.441347,1.365482,7.502001,1.0,1.182598,2.675532,6.023256,1


In [7]:
set(dframe['SLA'])

{0, 1, 2}

### Setting labels as dummies variables

In [16]:
dummies = ['SLA']

df_activ = pd.get_dummies(dframe.SLA, prefix='slo')
# Join the dummy variables to the main dataframe
dframe = pd.concat([dframe, df_activ], axis=1)
dframe.drop('SLA',inplace=True,axis=1)

In [17]:
df_activ.head()

,slo_0,slo_1,slo_2
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


### New labeled dataset

In [10]:
dframe.head()

,ellis-cpu.idle_perc.csv,ralf-load.avg_15_min.csv,homer-net.out_bytes_sec.csv,ellis-io.write_req_sec.csv,homestead-load.avg_1_min.csv,homer-load.avg_1_min.csv,ralf-cpu.idle_perc.csv,sprout-io.read_req_sec.csv,homestead-net.in_bytes_sec.csv,homer-disk.space_used_perc.csv,...,sprout-net.out_packets_sec.csv,ralf-cpu.system_perc.csv,ralf-mem.free_mb.csv,bono-io.read_req_sec.csv,bono-net.in_packets_sec.csv,homestead-cpu.wait_perc.csv,ellis-load.avg_1_min.csv,slo_0,slo_1,slo_2
0,7.155661,6.024735,2.956697,1.487579,1.493882,5.052356,8.517342,1.0,1.794528,1.0,...,1.441577,1.274112,7.513460,1.0,1.172696,4.207447,7.034884,0,1,0
1,7.728775,6.342756,2.946119,1.500860,1.367074,3.450262,8.582371,1.0,1.791270,1.0,...,1.428249,1.274112,7.508549,1.0,1.173083,3.058511,7.244186,0,1,0
2,7.410378,6.501766,2.962111,1.490639,1.387097,2.790576,8.517342,1.0,1.798901,1.0,...,1.440900,1.319797,7.511823,1.0,1.173375,2.723404,6.302326,0,1,0
3,7.176886,6.448763,2.950903,1.483509,1.273637,2.649215,8.634393,1.0,1.799224,1.0,...,1.426670,1.228426,7.500364,1.0,1.170270,2.723404,5.988372,0,1,0
4,7.346699,6.501766,2.954983,1.483509,1.233593,2.696335,8.322255,1.0,1.786141,1.0,...,1.441347,1.365482,7.502001,1.0,1.182598,2.675532,6.023256,0,1,0


### 2 - Visualization of the SLO classes using t-SNE plot

<img src="Figures/slo_tsne-generated_2000.png" />

### 3.   Description file were all the configuration is stored

### SLA description

<img src="SLA_def.png" />

### Configuration description in YAML

<img src="config_yml.png" />

### 5.    Set the randomness parameters in order to handle the number of deep leanring networks

with p = [9,9,8,8,7,7,6,6,5,5]

From lower layers to higher deep layers probabilities.

### 6.    perform and launch the program as a wrapper around Tensorflow

Launch tensorboar for visualization : tensorboard --logdir=/home/jaafar/Desktop/TNSM/logs

In [16]:
""" launch locally """

' launch locally '

### 7.    Show the GPU and CPU run and the process output


https://www.floydhub.com/heekof/projects/tnsm

### 8.    Show the results as a dataframe as well as other figures e.g. Confusion Matrix, 2D, 3D comparison etc


In [31]:
results =  pd.read_csv("output/stats_1511805998.85___Nmodels_3__MaxL_20_.csv", sep=';')

In [32]:
results.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
model,_1,_1,_2,_2,_3
batch_size,130,130,460,460,60
epoch,12,12,12,12,12
time,7.78696,2.96337,3.28126,2.11276,14.8051
info,info,info,info,info,info
acc_hist,"[0.45642857610114984, 0.45714286108102115, 0.4...","[0.45714286108102115, 0.45714286108102115, 0.4...","[0.45678570919803213, 0.45714285245963504]","[0.45714285245963504, 0.45714285245963504]","[0.45678571239113808, 0.45714285522699355, 0.4..."
acc,0.457143,0.457143,0.457143,0.457143,0.457143
loss_hist,"[1.0586897341268404, 1.0553072644131525, 1.055...","[1.0553676924535207, 1.0554491935031756, 1.055...","[nan, nan]","[nan, nan]","[nan, 1.098612070935113, 1.098612070935113, 1...."
loss,1.05554,1.05544,NaN,NaN,1.09861



<img src="Figures/camera_3D.png" />

### 9.    Show how the program selects the best ANN


<img src="Figures/10_best_Models_Gm.pdf" />


<img src="Figures/10_best.png" />

### 10. Show using decision tree algorithm the relevance and information gain of each ANN parameter in the overall results


<img src="Figures/relevance.png" />

## Meta-learning: Extrapolation using ML algorithm

### 11. Add a section on Meta-learning, leveraging the outputted results and extrapolating to find were the best ANN configuration can reside (This section is not incorporated in the nominal program run)

MetaLearning
<img src="Figures/MetaLearning" />


<img src="Figures/meta2.png" />

### 12. Show the provided offline documentation using MkDocs


launch mkdocs serve